In [ ]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# *Loading the data*

In [ ]:
LOCATION_CSV_TRAIN = "/content/gdrive/My Drive/Colab Notebooks/data-sea-ice/DRIFT_DATA_TRAIN.csv"
LOCATION_CSV_TEST = "/content/gdrive/My Drive/Colab Notebooks/data-sea-ice/DRIFT_DATA_TEST.csv"

In [ ]:
train = pd.read_csv(LOCATION_CSV_TRAIN)
test = pd.read_csv(LOCATION_CSV_TEST)

# Marking buoy ids found in the same year & day of year

Increasing a counter to mark groups of elements respecting the conditions stated above.

***Note***:

*  We used both test and train becasue we noticed that their were days that appeared only in test. Otherwise t values wouldn't reflect the true counts.

*  This will affect the split between validation and test if we won't account for the t values that came from test. We named those `bad_ts`

In [ ]:
t = 0
year_day_to_t = {}
train['t'] = 0 
test['t'] = 0
for year in sorted(list(train.year.unique())):
    for i in range(370):
        good_date = False
        if len(train[(train['year']==year) & (train['doy']==i)]) > 0:
            train.loc[(train.year==year) & (train.doy==i), 't'] = t
            good_date = True
        if len(test[(test['year']==year) & (test['doy']==i)]) > 0:
            test.loc[(test.year==year) & (test.doy==i), 't'] = t
            good_date = True
        if good_date:
            year_day_to_t[(year, i)] = t
            t += 1

train.head(10)

,year,month,day,doy,x_EASE,y_EASE,u_buoy,v_buoy,id_buoy,u_ERA5,v_ERA5,sic_CDR,h_cs2smos,h_piomas,d2c,t
0,1979,2,18,49,147.506958,138.582672,-0.797554,1.114740,1906,-6.704156,-0.321260,0.990195,NaN,3.189743,522.523298,0
1,1979,2,18,49,146.834778,120.509880,0.643200,0.368754,1913,-6.818630,-0.674205,0.966372,NaN,2.484009,412.767669,0
2,1979,2,18,49,130.993561,129.623672,-1.162420,0.243717,1914,-8.825469,1.123955,0.996022,NaN,2.474106,362.547379,0
3,1979,2,18,49,147.524719,157.382492,0.919766,0.025784,1918,-1.079951,-1.035410,0.982681,NaN,3.740522,381.025629,0
4,1979,2,19,50,147.470963,138.599823,0.380940,1.243485,1906,-2.169171,2.537787,0.990302,NaN,3.188522,521.535334,1
5,1979,2,19,50,180.349854,118.013527,1.387772,-0.253256,1911,2.680910,-0.295979,1.000000,NaN,2.574216,475.418633,1
6,1979,2,19,50,146.830490,120.509583,3.025445,1.076415,1913,0.551862,3.960332,1.000000,NaN,2.490376,412.761318,1
7,1979,2,19,50,130.940811,129.619873,1.409495,-0.041150,1914,-1.859920,1.140724,0.979121,NaN,2.480513,361.805709,1
8,1979,2,19,50,164.691742,110.154053,2.601760,2.154950,1925,1.245225,3.909907,0.992524,NaN,2.499613,404.686873,1
9,1979,2,20,51,197.865143,204.957596,-9.593610,-3.266865,1905,-2.742443,2.614781,0.964051,NaN,2.525601,367.538449,2


# Mapping points to a grid of various sizes (e.g.: 64)

We did try various values e.g. 128, 256 but we decided the best traid off between sparsity and size was at 64

In [ ]:
train['x_64'] = ((train.x_EASE * 64)/300).astype(int)
train['y_64'] = ((train.y_EASE * 64)/300).astype(int)

# Counts the number of overlaps 

Finds out the number of overlapping points in a grid cell for every grid size dimension.

We define an overlap when 2 data points have the same values for t, x_64 and y_64.


In [ ]:
threshold = 5
train['overlap_64'] = train.groupby(['t', 'x_64', 'y_64']).t.transform(np.size)
print("Overlap at 64 resolution ", len(train.loc[train.overlap_64 > threshold].t.unique()))

Overlap at 64 resolution  1129


In [ ]:
train.head(2)

,year,month,day,doy,x_EASE,y_EASE,u_buoy,v_buoy,id_buoy,u_ERA5,v_ERA5,sic_CDR,h_cs2smos,h_piomas,d2c,t,x_64,y_64,overlap_64
0,1979,2,18,49,147.506958,138.582672,-0.797554,1.114740,1906,-6.704156,-0.321260,0.990195,NaN,3.189743,522.523298,0,31,29,1
1,1979,2,18,49,146.834778,120.509880,0.643200,0.368754,1913,-6.818630,-0.674205,0.966372,NaN,2.484009,412.767669,0,31,25,1


# Names of the columns we are interested in

**List of features of interest**

`year, doy, x_EASE, y_EASE, u_buoy, v_buoy, u_ERA5, v_ERA5, sic_CDR, h_piomas, d2c`



In [ ]:
# used for scaling, creating the train and validation numpy 
good_columns = ['year', 'doy', 'x_EASE', 'y_EASE', 'u_buoy', 'v_buoy', 'u_ERA5', 'v_ERA5', 'sic_CDR', 'h_piomas', 'd2c']
# used to get the second input that gets fead to network
feature_2 = ['year', 'doy', 'x_EASE', 'y_EASE', 'u_ERA5', 'v_ERA5', 'sic_CDR', 'h_piomas', 'd2c']
# y we try to regress towards
outputs = ['u_buoy', 'v_buoy']

# Applies z-score for scaling the features

In [ ]:
column_norm = {}
for col in good_columns:
  column_norm[col] = {
      'std': train[col].std(),
      'mean': train[col].mean(),
  }
  train[col] = (train[col] - column_norm[col]['mean'])/column_norm[col]['std']

In [ ]:
for col in good_columns:
  test[col] = (test[col] - column_norm[col]['mean'])/column_norm[col]['std']

# Creating the main dataset for training as a numpy array

In [ ]:
GRID_SIZE = 64
number_ts = train.t.max()
print(f'Number of distinct t values found in dataset {number_ts}')
train_data = np.zeros((number_ts + 1, GRID_SIZE, GRID_SIZE, len(good_columns)), dtype=np.float32)

Number of distinct t values found in dataset 14590


In [ ]:
for t in range(train.t.max()):
  t_train = train.loc[train.t == t]
  for _, row in t_train.iterrows():
    if row['overlap_64'] == 1:
      train_data[int(t), int(row.x_64), int(row.y_64)] = row[good_columns].to_numpy()
    else:
      overlap = t_train.loc[(row.x_64 == t_train.x_64) & (row.y_64 == t_train.y_64)][good_columns].to_numpy()
      train_data[int(t), int(row.x_64), int(row.y_64)] = np.average(overlap, axis=0)
      

In [ ]:
train_data.shape

(14591, 64, 64, 11)

In [ ]:
class CNNDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataset, dataframe, test_dataframe, batch_size=32, shuffle=False, indexes=None):
        self.batch_size = batch_size
        self.df = dataframe
        self.t_max = dataframe.t.max()
        self.dataset = dataset
        self.test_df = test_dataframe
        if not indexes:
          self.indices = range(len(test_dataframe))
        else:
          self.indices = indexes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return (len(self.indices) // self.batch_size) + 1

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        X = self.__get_data(batch)
        return X

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):
        x1 = np.zeros((len(batch), 64, 64, 11), dtype=np.float32)
        x2 = np.zeros((len(batch), len(feature_2)), dtype=np.float32)
       
        for i, id in enumerate(batch):
          test_sample = self.test_df.iloc[id]
          x2[i] = test_sample[feature_2].to_numpy()
          state_t = test_sample.t
          if not test_sample.t in self.df.t:
            if test_sample.t + 1 in self.df.t:
              state_t += 1
            elif test_sample.t - 1 in self.df.t:
              state_t -= 1
            else:
              continue
          tvalues = self.df.loc[self.df.t == state_t]
          x1[i] = self.dataset[int(state_t)]

        return [x1, x2]

# Loading CNN models and using them to predict

In [ ]:
cnn_model = tf.keras.models.load_model("/content/gdrive/My Drive/Colab Notebooks/models/sea_ice_cnn_all")

In [ ]:
cnn_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 64, 64, 11)] 0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 49, 49, 32)   90144       input_7[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 24, 24, 32)   0           conv2d_12[0][0]                  
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 24, 24, 32)   128         max_pooling2d_9[0][0]            
____________________________________________________________________________________________

Calling the generators one for each split based on the indexes determined above

In [ ]:
cnn_data_gen = CNNDataGenerator(train_data, train, test, batch_size=128)
len(cnn_data_gen)

664

In [ ]:
predictions = cnn_model.predict(cnn_data_gen)
predictions.shape

(84870, 2)

In [ ]:
cnn_prediction_df = pd.DataFrame(data=predictions, columns=["u_buoy", "v_buoy"], index=None)
cnn_prediction_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/predictions/cnn_predictions_norm.csv", index=None)

In [ ]:
cnn_prediction_df["u_buoy"] = cnn_prediction_df["u_buoy"] * column_norm["u_buoy"]['std'] + column_norm["u_buoy"]['mean']
cnn_prediction_df["v_buoy"] = cnn_prediction_df["v_buoy"] * column_norm["v_buoy"]['std'] + column_norm["v_buoy"]['mean']
cnn_prediction_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/predictions/cnn_predictions.csv", index=None)

# Autoencoder

In [ ]:
def custom_mse(y_true, y_pred):
  mask = tf.not_equal(y_true, np.zeros((11), dtype=np.float32))
  y_true_masked = tf.boolean_mask(y_true, mask) 
  y_pred_masked = tf.boolean_mask(y_pred, mask)
  return tf.keras.losses.MSE(y_true=y_true_masked, y_pred=y_pred_masked)

In [ ]:
all_predictions = []
nd_batchsize = 5000
import gc
gc.collect()
for i in range(0, len(test), nd_batchsize):
  print (i)
  end = i+nd_batchsize
  if end > len(test):
    end = len(test)
  curr = range(i,end)
  print(len(curr))
  cnn_model = None
  ae_model = None 
  ae_data_gen = None
  gc.collect()

  ae_model = tf.keras.models.load_model("/content/gdrive/My Drive/Colab Notebooks/models/sea_ice_ae_cnn_all", custom_objects={'custom_mse': custom_mse})
  ae_data_gen = CNNDataGenerator(train_data, train, test, batch_size=64, indexes=curr)
  predictions_ae = ae_model.predict(ae_data_gen)
  
  with open('/content/gdrive/My Drive/Colab Notebooks/predictions/ae_pred_{}.npy'.format(i), 'wb') as f:
    np.save(f, predictions_ae[0])
  all_predictions.append(predictions_ae[0])


0
5000
5000
5000
10000
5000
15000
5000
20000
5000
25000
5000
30000
5000
35000
5000
40000
5000
45000
5000
50000
5000
55000
5000
60000
5000
65000
5000
70000
5000


In [ ]:
prediction_np = np.concatenate(all_predictions)
prediction_np.shape

(84736, 2)

In [ ]:
len(test)

84870

In [ ]:
ae_prediction_df = pd.DataFrame(data=ae_predictions[:,0], columns=["u_buoy", "v_buoy"], index=None)
ae_prediction_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/predictions/ae_predictions_norm.csv", index=None)
ae_prediction_df["u_buoy"] = ae_prediction_df["u_buoy"] * column_norm["u_buoy"]['std'] + column_norm["u_buoy"]['mean']
ae_prediction_df["v_buoy"] = ae_prediction_df["v_buoy"] * column_norm["v_buoy"]['std'] + column_norm["v_buoy"]['mean']
ae_prediction_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/predictions/ae_predictions.csv", index=None)

# Dense Network

In [ ]:
class DenseDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, test_dataframe, batch_size=32, shuffle=False):
        self.batch_size = batch_size
        self.df = dataframe
        self.t_max = dataframe.t.max()
        self.test_df = test_dataframe
        self.indices = range(len(test_dataframe))
        self.shuffle = shuffle
        self.on_epoch_end()
        self.max_t_count = dataframe.groupby('t')['id_buoy'].count().max()

    def __len__(self):
        return (len(self.indices) // self.batch_size) + 1

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        X = self.__get_data(batch)
        return X

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):
        x1 = np.zeros((len(batch), self.max_t_count, len(good_columns)), dtype=np.float32)
        x2 = np.zeros((len(batch_size), len(feature_2)), dtype=np.float32)
       
        for i, id in enumerate(batch):
          test_sample = self.test_df.iloc[id]
          x2[i] = test_sample[feature_2].to_numpy()
          state_t = test_sample.t
          x = np.zeros((self.max_t_count, len(good_columns)), dtype=np.float32)
          if not test_sample.t in self.df.t:
            if test_sample.t + 1 in self.df.t:
              state_t += 1
            elif test_sample.t - 1 in self.df.t:
              state_t -= 1
            else:
              continue
          tvalues = self.df.loc[self.df.t == state_t]
          x_np = tvalues[good_columns].to_numpy()
          x[:x_np.shape[0],:x_np.shape[1]] = x_np
          x1[i] = x

        return [x1, x2]


In [ ]:
cnn_model = None
ae_model = None
dense_model = tf.keras.models.load_model("/content/gdrive/My Drive/Colab Notebooks/models/sea_ice_naive_perm_60_all")
dense_model_data_gen = DenseDataGenerator(train, test, batch_size=128)

In [ ]:
dense_predictions = dense_model.predict(dense_model_data_gen)
dense_predictions.shape 84736

(84864, 2)

In [ ]:
dense_prediction_df = pd.DataFrame(data=dense_predictions, columns=["u_buoy", "v_buoy"], index=None)
dense_prediction_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/predictions/dense_predictions_norm.csv", index=None)
dense_prediction_df["u_buoy"] = dense_prediction_df["u_buoy"] * column_norm["u_buoy"]['std'] + column_norm["u_buoy"]['mean']
dense_prediction_df["v_buoy"] = dense_prediction_df["v_buoy"] * column_norm["v_buoy"]['std'] + column_norm["v_buoy"]['mean']
dense_prediction_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/predictions/dense_predictions.csv", index=None)